# GOOD Notebook to extract LMC and SMC Cadence


- Last update April 1 st 2021

- http://astro-lsst-01.astro.washington.edu:8080/ (baseline included)
- http://astro-lsst-01.astro.washington.edu:8081/  (rolling cadences included)
- http://astro-lsst-01.astro.washington.edu:8082/


In [344]:
# Check the version of MAF - the output should be version 2.4.2 or higher.
# 'unknown' means you're running from a git repo version,
# and have made changes to your local code

import lsst.sims.maf
lsst.sims.maf.__version__

'2.13.0.sims-93-g8bc2eb33+a14e41332a'

In [345]:
# import matplotlib to show plots inline.
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [346]:
# import our python modules
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metricBundles as metricBundles

from lsst.sims.maf.db import ResultsDb

In [347]:
import sys
print(sys.path)

['/Users/dagoret/MacOSX/GitHub/LSST/MyCadence/ExtractLMC', '/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/sims_w_2021_14/stack/miniconda3-py38_4.9.2-0.4.3/DarwinX86/sims_almanac/master-g021b69e146+616205b9df/python', '/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/sims_w_2021_14/stack/miniconda3-py38_4.9.2-0.4.3/DarwinX86/sims_downtimeModel/master-g55f72efa65+3e384ed16a/python', '/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/sims_w_2021_14/stack/miniconda3-py38_4.9.2-0.4.3/DarwinX86/sims_cloudModel/master-ge3724df529+632df0f48d/python', '/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/sims_w_2021_14/stack/miniconda3-py38_4.9.2-0.4.3/DarwinX86/sims_seeingModel/master-ga4bf72ea44+6e62bc95d5/python', '/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/sims_w_2021_14/stack/miniconda3-py38_4.9.2-0.4.3/DarwinX86/sims_featureScheduler/master-g5f42dc1c76+c0a10aa3f3/python', '/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/sims_w_2021_14/stack/miniconda3-py38_4.9.2-0.4.3/DarwinX86/sims_skybrightness_pre/2.13.0.sims-11-g9ab127b+

### Specify my other python path (my installation)

In [348]:
otherpythonpaths=['/Users/dagoret/anaconda3/lib/python37.zip', '/Users/dagoret/anaconda3/lib/python3.7', '/Users/dagoret/anaconda3/lib/python3.7/lib-dynload', '', '/Users/dagoret/.local/lib/python3.7/site-packages', '/Users/dagoret/anaconda3/lib/python3.7/site-packages', '/Users/dagoret/anaconda3/lib/python3.7/site-packages/aeosa', '/Users/dagoret/anaconda3/lib/python3.7/site-packages/IPython/extensions', '/Users/dagoret/.ipython']

In [349]:
for thepath in otherpythonpaths:
    sys.path.append(thepath) # go to parent dir

In [350]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Astronomy

In [351]:
gindex=0 # SELECT LMC

In [352]:
Galaxy_Name=["lmc","smc"]

In [353]:
Galaxy_Delta_RA=[10.75,5.33]
Galaxy_Delta_DEC=[9.17,3.083]

In [354]:
GName=Galaxy_Name[gindex]
galaxy_delta_ra=Galaxy_Delta_RA[gindex]
galaxy_delta_dec=Galaxy_Delta_DEC[gindex]

## astroquery

In [355]:
import astroquery
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle

In [356]:
from astroquery.simbad import Simbad
result_table = Simbad.query_object(GName)
result_table.pprint(show_unit=True)

MAIN_ID      RA        DEC    ... COO_QUAL COO_WAVELENGTH     COO_BIBCODE    
          "h:m:s"    "d:m:s"  ...                                            
-------- ---------- --------- ... -------- -------------- -------------------
NAME LMC 05 23 34.6 -69 45 22 ...        D              O 2003A&A...412...45P


In [357]:
Angle(result_table["RA"][0]+" hour")

<Angle 5.39294444 hourangle>

In [358]:
Angle(result_table["DEC"][0]+" degrees")

<Angle -69.75611111 deg>

In [359]:
galaxy_coord=SkyCoord(ra=result_table["RA"][0]+"hour",dec=result_table["DEC"][0]+ "degrees")
galaxy_coord

<SkyCoord (ICRS): (ra, dec) in deg
    (80.89416667, -69.75611111)>

In [360]:
galaxy_ra=galaxy_coord.ra.deg
galaxy_dec=galaxy_coord.dec.deg

In [361]:
print(galaxy_ra,galaxy_dec)

80.89416666666666 -69.75611111111111


# Config

## Database selection

In [362]:
dbindex=5 # SELECT

In [363]:
filters = ['u', 'g', 'r', 'i', 'z', 'y']
colors = {'u': 'cyan', 'g': 'g', 'r': 'y','i': 'r', 'z': 'm', 'y': 'k'}
Nfilt=len(filters)

In [364]:
database=["baseline2018a.db",\
          "baseline_v1.5_10yrs.db",\
          "baseline_2snaps_v1.5_10yrs.db",\
          "rolling_nm_scale1.0_nslice2_v1.7_10yrs.db",\
          "rolling_nm_scale1.0_nslice3_v1.7_10yrs.db",\
          "rolling_scale0.2_nslice3_v1.7_10yrs.db"
         ]

In [365]:
dbfileName=database[dbindex]

In [366]:
dbtag=dbfileName.split(".")[0]

In [367]:
dbtag

'rolling_scale0'

# Prepare access to database

## SQL Selection

In [368]:
raCol = 'fieldRA'
decCol = 'fieldDec'

In [369]:
sqlWhere = \
    'fieldRA < %f and ' \
    'fieldRA > %f and ' \
    'fieldDEC < %f and ' \
    'fieldDEC > %f  ' \
     % (galaxy_ra+ galaxy_delta_ra/2., galaxy_ra- galaxy_delta_ra/2.,galaxy_dec+galaxy_delta_dec/2.,galaxy_dec-galaxy_delta_dec/2)

In [370]:
sqlWhere

'fieldRA < 86.269167 and fieldRA > 75.519167 and fieldDEC < -65.171111 and fieldDEC > -74.341111  '

## From Tristan

In [371]:
RAMIN = 70.6 
RAMAX = 91.2 
DECMIN = -73.26 
DECMAX =-66.26

In [372]:
sqlWhere = \
    'fieldRA < %f and ' \
    'fieldRA > %f and ' \
    'fieldDEC < %f and ' \
    'fieldDEC > %f  ' \
     % (RAMAX, RAMIN,DECMAX,DECMIN)

In [373]:
sqlWhere

'fieldRA < 91.200000 and fieldRA > 70.600000 and fieldDEC < -66.260000 and fieldDEC > -73.260000  '

## Metric

In [374]:
dbfields=['observationId','night','observationStartMJD','fiveSigmaDepth','filter','airmass','skyBrightness','seeingFwhmEff','fieldId','fieldRA','fieldDec']

In [375]:
# This is very important to have the field number
metric=metrics.PassMetric(cols=dbfields)

##  Database to open

In [376]:
opsdb = db.OpsimDatabase('/Users/dagoret/DATA/Cadence/'+dbfileName)
outDir = 'output_galaxycadence'
resultsDb = db.ResultsDb(outDir=outDir)

## Slicer

In [377]:
slicer_uni=slicers.UniSlicer()
slicer_point = slicers.UserPointsSlicer(galaxy_ra, galaxy_dec, lonCol='fieldRA', latCol='fieldDec', latLonDeg='degrees')
nside = 512
slicer_heal = slicers.HealpixSlicer(nside=nside, lonCol=raCol, latCol=decCol)

Healpix slicer using NSIDE=512, approximate resolution 6.870973 arcminutes


# Run

## Bundle

- Bundles dict for all filter

In [378]:
bundles = {}
bundles[GName] = metricBundles.MetricBundle(metric, slicer_uni, sqlWhere)
print(bundles[GName].dbCols)

{'fieldRA', 'fieldDec', 'fieldId', 'seeingFwhmEff', 'night', 'observationStartMJD', 'fiveSigmaDepth', 'airmass', 'filter', 'skyBrightness', 'observationId'}


In [379]:
bgroup = metricBundles.MetricBundleGroup(bundles, opsdb, outDir=outDir,resultsDb=resultsDb)

In [380]:
bgroup.runAll()
bgroup.plotAll(closefigs=False)

Querying database SummaryAllProps with constraint fieldRA < 91.200000 and fieldRA > 70.600000 and fieldDEC < -66.260000 and fieldDEC > -73.260000   for columns ['fieldRA', 'fieldDec', 'fieldId', 'seeingFwhmEff', 'night', 'observationStartMJD', 'fiveSigmaDepth', 'airmass', 'filter', 'skyBrightness', 'observationId']
Found 1311 visits
Running:  ['lmc']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Plotting figures with "fieldRA < 91.200000 and fieldRA > 70.600000 and fieldDEC < -66.260000 and fieldDEC > -73.260000  " constraint now.
Plotting complete.


/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/sims_w_2021_14/stack/miniconda3-py38_4.9.2-0.4.3/DarwinX86/sims_maf/2.13.0.sims-93-g8bc2eb33+a14e41332a/python/lsst/sims/maf/db/resultsDb.py:279: UserWarning: Warning! Cannot save non-conforming summary statistic.
  warnings.warn('Warning! Cannot save non-conforming summary statistic.')


In [381]:
print(bundles[GName].dbCols)

{'fieldRA', 'fieldDec', 'fieldId', 'seeingFwhmEff', 'night', 'observationStartMJD', 'fiveSigmaDepth', 'airmass', 'filter', 'skyBrightness', 'observationId'}


In [382]:
print(len(bundles[GName].metricValues[0]))

1311


# Output

In [383]:
version="extr_v1"

In [384]:
pdfilename="cadence_"+ GName + "_" + dbtag + "_" + version + ".csv"

In [385]:
pdfilename

'cadence_lmc_rolling_scale0_extr_v1.csv'

In [386]:
df  = pd.DataFrame()

In [387]:
N=len(dbfields)

In [388]:
for index in np.arange(N):
    fieldname=dbfields[index]
    data=bundles[GName].metricValues[0][fieldname]
    df[fieldname]=data

In [389]:
df

,observationId,night,observationStartMJD,fiveSigmaDepth,filter,airmass,skyBrightness,seeingFwhmEff,fieldId,fieldRA,fieldDec
0,2788,4,59857.372150,22.135859,y,1.285771,18.369527,0.870150,-1,81.931712,-69.094794
1,2789,4,59857.372603,22.135031,y,1.327240,18.318414,0.846100,-1,77.179586,-71.354700
2,2790,4,59857.373052,22.185094,y,1.276681,18.352536,0.826612,-1,73.226252,-68.599593
3,2791,4,59857.373503,22.226043,y,1.238969,18.384254,0.811874,-1,77.842629,-66.428590
4,3598,5,59858.343894,22.184296,y,1.337607,18.280521,0.793993,-1,71.700652,-71.812836
...,...,...,...,...,...,...,...,...,...,...,...
1306,1890799,3411,63264.080365,24.321436,g,1.281804,22.139102,1.230974,-1,86.991217,-68.931094
1307,1895925,3419,63272.059658,23.246838,u,1.278964,22.613479,1.213124,-1,84.004985,-68.630787
1308,1895926,3419,63272.060121,23.275625,u,1.254773,22.636920,1.199304,-1,90.722595,-67.403802
1309,1895958,3419,63272.076411,24.105331,g,1.287915,22.136049,1.498725,-1,84.004985,-68.630787


In [390]:
df.to_csv(pdfilename)

In [391]:
!ls -l -t

total 1984
-rw-r--r--  1 dagoret  staff  197486 Apr  1 22:03 cadence_lmc_rolling_scale0_extr_v1.csv
drwxr-xr-x  5 dagoret  staff     160 Apr  1 22:03 output_galaxycadence
-rw-r--r--@ 1 dagoret  staff   40572 Apr  1 22:03 DumpLMCCadence.ipynb
-rw-r--r--  1 dagoret  staff  185700 Apr  1 22:02 cadence_lmc_rolling_nm_scale1_extr_v1.csv
-rw-r--r--@ 1 dagoret  staff   39394 Apr  1 22:02 ExtractLMCCadence.ipynb
-rw-r--r--  1 dagoret  staff  193420 Apr  1 21:20 cadence_lmc_baseline_2snaps_v1_extr_v1.csv
-rw-r--r--  1 dagoret  staff  197231 Apr  1 21:18 cadence_lmc_baseline_v1_extr_v1.csv
-rw-r--r--  1 dagoret  staff  103980 Apr  1 21:12 cadence_lmc_baseline2018a_extr_v1.csv
